Library


In [79]:
import math
import pickle
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import scipy.fftpack
import librosa
import soundfile as sf
from scipy.signal import resample_poly
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.tree import DecisionTreeClassifier, export_text
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

Main Path


In [27]:
# MAIN_PATH = r"C:\Users\Lulay\Documents\GitHub\Dasar-Kecerdasan-Artificial_Tugas-Besar\Dataset"
MAIN_PATH = r"D:\Kuliah\Matkul\Semester 4\DASAR KECERDASAN ARTIFICIAL (DKA)\[2] Tugas\[3] Tugas Besar\Dasar-Kecerdasan-Artificial_Tugas-Besar\Dataset"

Konstanta


In [28]:
SAMPLE_RATE = 48000
BATCH_SIZE = 32
SEGMENT_DURATION = int(1 * SAMPLE_RATE)
OVERLAP_DURATION = int(0.5 * SAMPLE_RATE)
RANDOM_STATE = 21
BUFFER_SIZE = 1000

Random Seed


In [29]:
RANDOM_SEED = 21

try:
    from sklearn.utils import check_random_state
    random_state = check_random_state(RANDOM_SEED)
except ImportError:
    pass

# Prapemrosesan


In [30]:
def load_file_audio(path):
    audio, sr = sf.read(path)
    return np.array(audio), sr

def prapemrosesan_downmixing(audio):
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    return audio.astype(np.float32)

def prapemrosesan_resampling(audio, sr):
    if sr == SAMPLE_RATE:
        return audio.copy(), SAMPLE_RATE
    
    ratio = SAMPLE_RATE / sr
    n_samples = int(np.round(len(audio) * ratio))
    
    x_old = np.linspace(0, 1, len(audio))
    x_new = np.linspace(0, 1, n_samples)
    return np.interp(x_new, x_old, audio), SAMPLE_RATE

def prapemrosesan_padding(audio):
    if np.mod(audio.shape[0], SEGMENT_DURATION) != 0:
        padding = SEGMENT_DURATION - (audio.shape[0] % SEGMENT_DURATION)
        audio = np.pad(audio, (0, padding))
    return audio

def prapemrosesan_splitting(audio):
    num_segments = int(np.floor((len(audio) - SEGMENT_DURATION) / OVERLAP_DURATION)) + 1
    segments = []

    for i in range(num_segments):
        start = int(i * OVERLAP_DURATION)
        end = int(start + SEGMENT_DURATION)
        segment = audio[start:end]
        if len(segment) < SEGMENT_DURATION:
            segment = np.pad(segment, (0, SEGMENT_DURATION - len(segment)), mode='constant')
        segments.append(segment)

    return np.array(segments)

# Ekstraksi Fitur


In [31]:
def get_rms(segment):
    return np.sqrt(np.mean(segment ** 2))

def get_zcr(segment):
    return np.sum(np.abs(np.diff(np.signbit(segment)))) / (len(segment) / SAMPLE_RATE)

def get_lms(segment):
    mel_spec = librosa.feature.melspectrogram(y=segment, sr=SAMPLE_RATE)
    return np.mean(mel_spec)

# Dataset


Raw


In [32]:
with open(f"{MAIN_PATH}/dataset_rmszcrlms.pkl", "rb") as f:
    dataset = pickle.load(f)

In [33]:
len(dataset)

865

Normalisasi, Split, dan Batching


In [56]:
dataset[0][1:]

(0.0,
 1.0,
 2,
 0.011059444,
 0.23255173703457446,
 array([[2.22129905e-13, 1.03116427e-09, 1.56951060e-07, ...,
         5.70585842e-13, 1.14141976e-07, 3.86931497e-05],
        [1.73044982e-13, 1.74463599e-09, 6.80466954e-08, ...,
         5.17760639e-13, 1.17636276e-07, 3.98712073e-05],
        [1.94202402e-13, 1.31313471e-09, 1.05966667e-08, ...,
         2.38242260e-12, 1.35159581e-07, 4.56374873e-05],
        ...,
        [2.20867216e-16, 3.81760608e-16, 5.07324878e-16, ...,
         5.14235058e-14, 2.11715911e-09, 6.91677428e-07],
        [9.88679711e-21, 1.61993748e-18, 5.16642328e-18, ...,
         3.58666228e-16, 2.08386641e-09, 6.80778953e-07],
        [4.95487801e-22, 1.56139436e-20, 6.90402098e-21, ...,
         3.08383758e-16, 2.06350248e-09, 6.74107753e-07]], dtype=float32))

In [59]:
X = [data[6] for data in dataset]
original_labels = [int(data[3]) for data in dataset]
y = [1 if label in (0, 1) else 2 for label in original_labels]
X = np.array(X)

In [61]:
X[0].shape

(128, 94)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE, stratify=y_test)

In [63]:
print(type(X_train[0]))
print(np.array(X_train[0]).shape)


<class 'numpy.ndarray'>
(128, 94)


In [64]:
X_train_reshaped = np.expand_dims(X_train, axis=1)
X_test_reshaped = np.expand_dims(X_test, axis=1)
X_val_reshaped = np.expand_dims(X_val, axis=1)

In [65]:
display(
    X_train_reshaped.shape,
    X_test_reshaped.shape,
    X_val_reshaped.shape
)

(692, 1, 128, 94)

(86, 1, 128, 94)

(87, 1, 128, 94)

# CNN


Inisialisasi


In [66]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(256, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

training


In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train_reshaped_tensor = torch.tensor(X_train_reshaped, dtype=torch.float32).reshape(-1, 1, 128, 94).to(device)
X_val_reshaped_tensor = torch.tensor(X_val_reshaped, dtype=torch.float32).reshape(-1, 1, 128, 94).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

num_epochs = 100
batch_size = 128

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    y_train_preds = []
    y_train_true = []
    
    for i in range(0, len(X_train_reshaped_tensor), batch_size):
        inputs = X_train_reshaped_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        y_train_preds.extend(predicted.cpu().numpy())
        y_train_true.extend(targets.cpu().numpy())

    train_accuracy = accuracy_score(y_train_true, y_train_preds)
    train_f1_score = f1_score(y_train_true, y_train_preds, average='weighted')

    model.eval()
    y_val_preds = []
    y_val_true = []
    
    with torch.no_grad():
        for i in range(0, len(X_val_reshaped_tensor), batch_size):
            inputs = X_val_reshaped_tensor[i:i+batch_size]
            targets = y_val_tensor[i:i+batch_size].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_val_preds.extend(predicted.cpu().numpy())
            y_val_true.extend(targets.cpu().numpy())

    val_accuracy = accuracy_score(y_val_true, y_val_preds)
    val_f1_score = f1_score(y_val_true, y_val_preds, average='weighted')

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(X_train_reshaped_tensor):.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1_score:.4f}, "
          f"Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1_score:.4f}")


Epoch [1/100], Loss: 0.0175, Train Accuracy: 0.5549, Train F1 Score: 0.6349, Val Accuracy: 0.7471, Val F1 Score: 0.7471
Epoch [2/100], Loss: 0.0119, Train Accuracy: 0.7283, Train F1 Score: 0.7240, Val Accuracy: 0.6782, Val F1 Score: 0.6155
Epoch [3/100], Loss: 0.0062, Train Accuracy: 0.7124, Train F1 Score: 0.6934, Val Accuracy: 0.6897, Val F1 Score: 0.6133
Epoch [4/100], Loss: 0.0052, Train Accuracy: 0.6908, Train F1 Score: 0.6694, Val Accuracy: 0.7011, Val F1 Score: 0.6330
Epoch [5/100], Loss: 0.0049, Train Accuracy: 0.7240, Train F1 Score: 0.7107, Val Accuracy: 0.7126, Val F1 Score: 0.6823
Epoch [6/100], Loss: 0.0048, Train Accuracy: 0.7355, Train F1 Score: 0.7201, Val Accuracy: 0.7011, Val F1 Score: 0.7050
Epoch [7/100], Loss: 0.0047, Train Accuracy: 0.7283, Train F1 Score: 0.7146, Val Accuracy: 0.3793, Val F1 Score: 0.2086
Epoch [8/100], Loss: 0.0047, Train Accuracy: 0.7486, Train F1 Score: 0.7350, Val Accuracy: 0.3793, Val F1 Score: 0.2086
Epoch [9/100], Loss: 0.0045, Train Accur

In [85]:
X_train_reshaped = X_train_reshaped.reshape(X_train_reshaped.shape[0], -1)
X_val_reshaped = X_val_reshaped.reshape(X_val_reshaped.shape[0], -1)



In [84]:
X_train_reshaped.shape

(692, 12032)

# LGBM


Inisialisasi


In [86]:
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)

param_grid_lgbm = {
    'n_estimators': [50, 100, 200],
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [-1, 10, 20]
}

training


In [89]:
param_grid = list(ParameterGrid(param_grid_lgbm))
best_score_lgbm = 0
best_model_lgbm = None

for i, params in enumerate(param_grid, 1):
    lgbm_model = lgbm_model.set_params(**params)
    lgbm_model.fit(X_train_reshaped, y_train)

    y_pred = lgbm_model.predict(X_val_reshaped)
    acc = accuracy_score(y_val, y_pred)

    print(f"[{i}/{len(param_grid)}] Params: {params} | Val Accuracy: {acc:.4f}")

    if acc > best_score_lgbm:
        best_score_lgbm = acc
        best_model_lgbm = lgbm_model

[1/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 50, 'num_leaves': 31} | Val Accuracy: 0.8621
[2/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 50, 'num_leaves': 50} | Val Accuracy: 0.8621
[3/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 50, 'num_leaves': 100} | Val Accuracy: 0.8621
[4/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 31} | Val Accuracy: 0.8621
[5/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 50} | Val Accuracy: 0.8621
[6/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 100} | Val Accuracy: 0.8621
[7/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 31} | Val Accuracy: 0.8736
[8/81] Params: {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 50} | Val Accuracy: 0.8736
[9/81] Params: {'learning_rate': 0.01, 'max_depth': -1, '

# Decision Tree


Inisialisasi


In [ ]:
dt_model = DecisionTreeClassifier(random_state=RANDOM_STATE)

param_grid_dt = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

training


In [ ]:
grid_dt = GridSearchCV(dt_model, param_grid_dt, cv=3, scoring='accuracy')
grid_dt.fit(X_train_reshaped, y_train)
best_dt_model = grid_dt.best_estimator_

## Random Forest

Inisialisasi


In [ ]:
rf_model = RandomForestClassifier(random_state=RANDOM_STATE)

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

training


In [ ]:
grid_rf = GridSearchCV(rf_model, param_grid_rf, cv=3, scoring='accuracy')
grid_rf.fit(X_train_reshaped, y_train)

xg_model = grid_rf.best_estimator_

## XGBoost

Inisialisasi


In [ ]:
xg_model = XGBClassifier(random_state=RANDOM_STATE)

param_grid_xg = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

training


In [ ]:
grid_xg = GridSearchCV(xg_model, param_grid_xg, cv=3, scoring='accuracy')
grid_xg.fit(X_train_reshaped, y_train)

xg_model = grid_xg.best_estimator_

ValueError: 
All the 81 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Fathan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Fathan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\Fathan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-pa

## Gradient Boosting

Inisialisasi


In [ ]:
gb_model = GradientBoostingClassifier(random_state=RANDOM_STATE)

param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10]
}

training


In [ ]:
grid_gb = GridSearchCV(gb_model, param_grid_gb, cv=3, scoring='accuracy')
grid_gb.fit(X_train_reshaped, y_train)
gb_model = grid_gb.best_estimator_

KeyboardInterrupt: 

## Logistic Regression

Inisialisasi


In [ ]:
lr_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)

param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}


training


In [ ]:
grid_lr = GridSearchCV(lr_model, param_grid_lr, cv=3, scoring='accuracy')
grid_lr.fit(X_train_reshaped, y_train)

lr_model = grid_lr.best_estimator_

## K-Nearest Neighbors

Inisialisasi


In [ ]:
knn_model = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

training


In [ ]:
grid_knn = GridSearchCV(knn_model, param_grid_knn, cv=3, scoring='accuracy')
grid_knn.fit(X_train_reshaped, y_train)
knn_model = grid_knn.best_estimator_

## Support Vector Machine (SVM)

Inisialisasi


In [ ]:
svm_model = SVC(random_state=RANDOM_STATE)

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

training


In [ ]:
grid_svm = GridSearchCV(svm_model, param_grid_svm, cv=3, scoring='accuracy')
grid_svm.fit(X_train_reshaped, y_train)
svm_model = grid_svm.best_estimator_

# Conformer


Inisialisasi


In [ ]:
# config = ConformerConfig(
#     hidden_size=144,
#     num_attention_heads=4,
#     intermediate_size=576,
#     conv_kernel_size=32,
#     num_hidden_layers=8,
#     input_feat_per_channel=n_mels,
#     input_channels=1,
#     max_position_embeddings=time_steps
# )

# conformer_model = TFConformerModel(config)

# input_layer = tf.keras.layers.Input(shape=(n_mels, time_steps))
# expand_dim = tf.keras.layers.Reshape((1, n_mels, time_steps))(input_layer)
# conv_proj = tf.keras.layers.Conv2D(
#     filters=config.hidden_size,
#     kernel_size=(3, 3),
#     padding='same',
#     activation='relu'
# )(expand_dim)
# squeeze_dim = tf.keras.layers.Reshape((n_mels, time_steps, config.hidden_size))(conv_proj)
# conformer_output = conformer_model(squeeze_dim).last_hidden_state
# gap = tf.keras.layers.GlobalAveragePooling1D()(conformer_output)
# output = tf.keras.layers.Dense(num_classes, activation='softmax')(gap)

# conformer_model = tf.keras.Model(inputs=input_layer, outputs=output)

training


# Evaluasi


CNN


LGBM


In [ ]:
y_pred = lgbm_model.predict(X_val_reshaped)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

y_pred = lgbm_model.predict(X_test_reshaped)
print("\n")
print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

NotFittedError: Estimator not fitted, call fit before exploiting the model.

Descision Tree


In [ ]:
y_pred = dt_model.predict(X_val_reshaped)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

y_pred = dt_model.predict(X_test_reshaped)

print("\n")
print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

In [ ]:
feature_names = ['RMS', 'ZCR']
tree_rules = export_text(dt_model, feature_names=feature_names)
print("Extracted Decision Rules:")
print(tree_rules)

## Random Forest

In [ ]:
y_pred_rf = rf_model.predict(X_val_reshaped)
print("Random Forest Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf)}")
print(f"Precision: {precision_score(y_val, y_pred_rf)}")
print(f"Recall: {recall_score(y_val, y_pred_rf)}")
print(f"F1 Score: {f1_score(y_val, y_pred_rf)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred_rf)}")

y_pred_rf = rf_model.predict(X_test_reshaped)
print("\nRandom Forest Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")
print(f"Precision: {precision_score(y_test, y_pred_rf)}")
print(f"Recall: {recall_score(y_test, y_pred_rf)}")
print(f"F1 Score: {f1_score(y_test, y_pred_rf)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_rf)}")

## XGBoost

In [ ]:
y_pred_xg = xg_model.predict(X_val_reshaped)
print("XGBoost Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred_xg)}")
print(f"Precision: {precision_score(y_val, y_pred_xg)}")
print(f"Recall: {recall_score(y_val, y_pred_xg)}")
print(f"F1 Score: {f1_score(y_val, y_pred_xg)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred_xg)}")

y_pred_xg = xg_model.predict(X_test_reshaped)
print("\nXGBoost Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xg)}")
print(f"Precision: {precision_score(y_test, y_pred_xg)}")
print(f"Recall: {recall_score(y_test, y_pred_xg)}")
print(f"F1 Score: {f1_score(y_test, y_pred_xg)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_xg)}")

## Gradient Boosting

In [ ]:
y_pred_gb = gb_model.predict(X_val_reshaped)
print("Gradient Boosting Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred_gb)}")
print(f"Precision: {precision_score(y_val, y_pred_gb)}")
print(f"Recall: {recall_score(y_val, y_pred_gb)}")
print(f"F1 Score: {f1_score(y_val, y_pred_gb)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred_gb)}")

y_pred_gb = gb_model.predict(X_test_reshaped)
print("\nGradient Boosting Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb)}")
print(f"Precision: {precision_score(y_test, y_pred_gb)}")
print(f"Recall: {recall_score(y_test, y_pred_gb)}")
print(f"F1 Score: {f1_score(y_test, y_pred_gb)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_gb)}")


## Logistic Regression

In [ ]:
y_pred_lr = lr_model.predict(X_val_reshaped)
print("Logistic Regression Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred_lr)}")
print(f"Precision: {precision_score(y_val, y_pred_lr)}")
print(f"Recall: {recall_score(y_val, y_pred_lr)}")
print(f"F1 Score: {f1_score(y_val, y_pred_lr)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred_lr)}")

y_pred_lr = lr_model.predict(X_test_reshaped)
print("\nLogistic Regression Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr)}")
print(f"Precision: {precision_score(y_test, y_pred_lr)}")
print(f"Recall: {recall_score(y_test, y_pred_lr)}")
print(f"F1 Score: {f1_score(y_test, y_pred_lr)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_lr)}")


## K-Nearest Neighbors

In [ ]:
y_pred_knn = knn_model.predict(X_val_reshaped)
print("KNN Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred_knn)}")
print(f"Precision: {precision_score(y_val, y_pred_knn)}")
print(f"Recall: {recall_score(y_val, y_pred_knn)}")
print(f"F1 Score: {f1_score(y_val, y_pred_knn)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred_knn)}")

y_pred_knn = knn_model.predict(X_test_reshaped)
print("\nKNN Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn)}")
print(f"Precision: {precision_score(y_test, y_pred_knn)}")
print(f"Recall: {recall_score(y_test, y_pred_knn)}")
print(f"F1 Score: {f1_score(y_test, y_pred_knn)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_knn)}")


## Support Vector Machine (SVM)

In [ ]:
y_pred_svm = svm_model.predict(X_val_reshaped)
print("SVM Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred_svm)}")
print(f"Precision: {precision_score(y_val, y_pred_svm)}")
print(f"Recall: {recall_score(y_val, y_pred_svm)}")
print(f"F1 Score: {f1_score(y_val, y_pred_svm)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred_svm)}")

y_pred_svm = svm_model.predict(X_test_reshaped)
print("\nSVM Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm)}")
print(f"Precision: {precision_score(y_test, y_pred_svm)}")
print(f"Recall: {recall_score(y_test, y_pred_svm)}")
print(f"F1 Score: {f1_score(y_test, y_pred_svm)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_svm)}")


Conformer


# Visualisasi Hasil

In [ ]:
def load_and_plot(path, list_result, final_time):
    file_name = os.path.basename(path)
    audio, sr = librosa.load(path, sr=SAMPLE_RATE)
    time_axis = np.linspace(0, len(audio) / sr, len(audio))

    plt.figure(figsize=(20, 4))
    plt.plot(time_axis, audio, label="Amplitude")
    plt.title(f"Audio waveform with loud segments marked - {file_name}") 
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

    for i in enumerate(list_result):
        start_time = i[1][0]
        end_time = i[1][1]
        plt.axvspan(start_time, end_time, color='red', alpha=0.3)

    plt.xticks(np.arange(0, final_time, 0.5))
    plt.grid(which='both', alpha=0.5)
    plt.xlim(0, final_time)
    plt.ylim(-1, 1)
    plt.tight_layout()
    plt.legend()
    plt.show()

### LGBM

In [ ]:
def single_inference(path):
    audio, sr = load_file_audio(path)
    audio = prapemrosesan_downmixing(audio)
    audio, sr = prapemrosesan_resampling(audio, sr)
    audio = prapemrosesan_padding(audio)
    segments = prapemrosesan_splitting(audio)
    
    list_result = []
    for index, segment in enumerate(segments):
        rms = get_rms(segment)
        zcr = get_zcr(segment)
        X = np.array([rms, zcr])
        X = scl.transform(X.reshape(1, -1))

        y_pred = lgbm_model.predict(X)
        if y_pred == 0:
            list_result.append(([index*0.5, index*0.5 + 1]))
    return list_result

In [ ]:
# for file in os.listdir(f"{MAIN_PATH}/noise-audio-data"):
#     list_result = single_inference(f"{MAIN_PATH}/noise-audio-data/{file}")
#     print(list_result)
#     load_and_plot(f"{MAIN_PATH}/noise-audio-data/{file}", list_result, 5)  

In [ ]:
# for file in os.listdir(f"{MAIN_PATH}/xeno-canto"):
#     list_result = single_inference(f"{MAIN_PATH}/xeno-canto/{file}")
#     print(list_result)
#     load_and_plot(f"{MAIN_PATH}/xeno-canto/{file}", list_result, 25)  